In [1]:
from superneuromat.neuromorphicmodel import NeuromorphicModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field


# from: https://stackoverflow.com/a/21894086/2712730
class bidict(dict):
    """Creates a dictionary that supports reverse lookups via the .inverse attribute.

    Args:
        dict (dict): The original dictionary.

    Properties:
        inverse (dict): A dictionary that maps values to keys.
    """
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value, []).append(key)

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key)
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value, []).append(key)

    def __delitem__(self, key):
        self.inverse.setdefault(self[key], []).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]:
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)


def train_test_split_indices(papers, test_size=0.2, rng=None) -> tuple[np.ndarray[int], np.ndarray[int]]:
    """
    Splits a list of papers into train and test indices.

    Parameters
    ----------
    papers : iterable of papers with ids
        List or dict of papers to split.
    test_size : float, optional
        if < 1, proportion of papers to reserve for testing, by default 0.2
        if > 1, number of papers to reserve for testing
    rng : int, optional
        Random state for the random number generator, default uses numpy's random

    Returns
    -------
    train_indices : list of int
        List of indices for the training set.
    test_indices : list of int
        List of indices for the testing set.
    """
    n = papers if isinstance(papers, int) else len(papers)
    if test_size < 1:
        test_size = int(np.floor(test_size * n))  # number of papers in test
    if rng is None:
        rng = np.random.default_rng()  # setup rng
    elif isinstance(rng, int):
        rng = np.random.default_rng(rng)

    if isinstance(papers, int):
        indices = np.arange(n)  # generate indices
    elif isinstance(papers, (list, tuple)):
        indices = papers  # assume papers is a list of indices
    else:  # assume papers is a dict or mapping of papers with a .values() method
        indices = [paper.idx for paper in papers.values()]  # grab indices from dict entries
    indices = np.asarray(indices)

    # shuffle and split
    rng.shuffle(indices)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return train_indices, test_indices


In [2]:
rng = np.random.default_rng(2024)

### Load CiteSeer dataset

In [3]:
@dataclass
class Paper:
    idx: str  # Paper ID
    label: str  # Paper category/topic
    features: tuple[bool, ...] = ()  # binary features
    citations: list[str] = field(default_factory=list)  # IDs of papers cited by this paper

papers = {}
missing = set()

# Load in training data
content = pd.read_csv("data/citeseer/citeseer.content", sep="\t", header=None, dtype=object)
citations = pd.read_csv("data/citeseer/citeseer.cites", sep="\t", header=None, dtype=object)

labels = set()  # set of unique labels

for paper in content.itertuples(index=False):  # create papers from data
    idx = paper[0]
    features = tuple([int(feature) for feature in paper[1:-1]])  # parse features
    papers[idx] = Paper(idx, paper[-1], features)  # create paper object
    labels.add(paper[-1])  # label is the last column. add to set of labels

for paper_idx, citation in citations.itertuples(index=False):
    try:  # parse citations
        if citation not in papers:
            missing.add(citation)  # if citation is missing, add to missing list
            continue
        papers[paper_idx].citations.append(citation)
    except KeyError:
        missing.add(paper_idx)  # if paper is missing, add to missing list

print(f"Loaded {len(papers)} papers and skipped {len(missing)} broken references.")

labels  # show labels set

Loaded 3312 papers and skipped 15 broken references.


{'AI', 'Agents', 'DB', 'HCI', 'IR', 'ML'}

#### Create train / test split  
Create two lists of IDs

In [4]:
train_idxs, test_idxs = train_test_split_indices(papers, test_size=0.2, rng=rng)

#### Build the network

In [5]:
# Initialize our model
model = NeuromorphicModel()

# Create our output neurons, set threshold very high so that we control when they spike during training.
# dict mapping {category: neuron_id}
lbl_threshold = 2.0
strong_connection = 5.0
weak_connection = 1.0
unknown_connection = 0.00001

lbl_neurons = bidict({label: model.create_neuron(threshold=lbl_threshold).idx for label in labels})

# Create our input neurons, one for each pixel of the image resolution.
paper_neurons = {}  # dict mapping {paper_id: neuron_id}

# make a neuron for each paper
for paper in papers.values():
    paper_neurons[paper.idx] = neuron_id = model.create_neuron().idx

    if paper.idx in train_idxs:
        # Make an explicitly STRONG synapse connecting the input to the output
        output_id = lbl_neurons[paper.label]  # (training paper to topic)
        model.create_synapse(neuron_id, output_id, weight=strong_connection, stdp_enabled=False, delay=1)
        model.create_synapse(output_id, neuron_id, weight=strong_connection, stdp_enabled=False, delay=1)
    else:  # test set, connect this input neuron to all output neurons
        # Connect our input neuron to output neurons (test paper to topic)
        for output_id in lbl_neurons.values():
            # Randomize initial weight
            weight1 = (rng.uniform(-1, 1)) * unknown_connection
            weight2 = (rng.uniform(-1, 1)) * unknown_connection
            # Make a synapse connecting the input to the output
            model.create_synapse(neuron_id, output_id, weight=weight1, stdp_enabled=True, delay=1)
            model.create_synapse(output_id, neuron_id, weight=weight2, stdp_enabled=True, delay=1)

# connect papers by their citations (paper to paper connections)
for paper in papers.values():
    for citation in paper.citations:
        model.create_synapse(paper_neurons[paper.idx], paper_neurons[citation], weight=weak_connection, stdp_enabled=True, delay=1)
        model.create_synapse(paper_neurons[citation], paper_neurons[paper.idx], weight=weak_connection, stdp_enabled=True, delay=1)

#### loop through our dataset and add spikes

In [6]:
timestep = 0
train_idxs_augmented = np.append(train_idxs, train_idxs)
for idx in train_idxs_augmented:
    paper = papers[idx]  # get paper by id
    # Add a spike that will exceed the threshold for the respective label neuron
    model.add_spike(timestep + 1, lbl_neurons[paper.label], strong_connection)
    # Add spikes to the paper
    model.add_spike(timestep, paper_neurons[paper.idx], strong_connection)
    timestep += 1

### Setup the model and perform a training pass

In [7]:
# Set up our stdp, only one timestep because we only want it looking at the results
# of what our input layer does to our output layer
model.stdp_setup(Apos=[1e-4, 1e-4], Aneg=[-1e-5, -1e-5], negative_update=True, positive_update=True)
# model.setup()

print(f"Your model has {model.num_neurons} neurons and {model.num_synapses} synapses.\n{timestep} time steps will be simulated.")

from tqdm.notebook import tqdm

# Simulate
with tqdm(total=timestep) as pbar:
    model.simulate(time_steps=timestep, callback=lambda _s, _t, _n: pbar.update())

Your model has 3318 neurons and 22674 synapses.
5300 time steps will be simulated.


  0%|          | 0/5300 [00:00<?, ?it/s]

### inference

In [8]:
def blank_copy(model):
    new = NeuromorphicModel()
    new.neuron_states = model.neuron_states.copy()
    new.neuron_refractory_periods_state = model.neuron_refractory_periods_state.copy()
    new.neuron_thresholds = model.neuron_thresholds.copy()
    new.neuron_leaks = model.neuron_leaks.copy()
    new.neuron_reset_states = model.neuron_reset_states.copy()
    new.neuron_refractory_periods = model.neuron_refractory_periods.copy()
    new.pre_synaptic_neuron_ids = model.pre_synaptic_neuron_ids.copy()
    new.post_synaptic_neuron_ids = model.post_synaptic_neuron_ids.copy()
    new.synaptic_weights = model.synaptic_weights.copy()
    new.synaptic_delays = model.synaptic_delays.copy()
    new.enable_stdp = model.enable_stdp.copy()
    return new

In [9]:
seed = 10022

In [ ]:
seed += 1
print(f"Seed: {seed}")
rng = np.random.default_rng(seed)
model2 = blank_copy(model)
model2.stdp = False
nlabels = len(labels)
# model2.enable_stdp = np.zeros(model.num_neurons)

model2.neuron_thresholds[:nlabels] = [99999] * nlabels

# paper = random.choice(list(papers.values()))  # pick a random paper
# paper = papers[random.choice(train_idxs)]  # pick a random paper from training set
paper = papers[rng.choice(test_idxs)]  # pick a random paper from testing set

print(f"Paper: {paper.idx}\tCategory: {paper.label}")

spike_n = 2

for t in range(spike_n):
    model2.add_spike(t, paper_neurons[paper.idx], strong_connection)

model2.simulate(spike_n + 1)

output_spikes = np.sum(model2.ispikes[-(spike_n + 1):, :nlabels], axis=0)

spiked_ids = {idx: lbl_neurons.inverse[idx][0]
              for idx, spiked in enumerate(output_spikes) if spiked}
if spiked_ids:
    print(f"Spiked categories:")
    for idx, category in spiked_ids.items():
        print(f"\t{idx}\t{category}")
else:
    print("No category spiked")

print('=' * 10)

lbl_by_threshold = sorted((enumerate(model2.neuron_states[:nlabels])), key=lambda x: x[1], reverse=True)
for i, v in lbl_by_threshold:
    category = lbl_neurons.inverse[i][0]
    print(f"{i}\t{v: 5.5f}\t{category}")
print(model2.neuron_states[:nlabels])
print(model2.neuron_thresholds[:nlabels])
for ts in model2.spike_train:
    print(ts[:nlabels])

Seed: 10030
Paper: terrillon98automatic	Category: ML
Spiked categories:
	0	ML
	1	Agents
	2	HCI
	3	AI
	4	IR
	5	DB
0	 0.00000	ML
1	 0.00000	Agents
2	 0.00000	HCI
3	 0.00000	AI
4	 0.00000	IR
5	 0.00000	DB
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[6, 6, 6, 6, 6, 6]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0.]
[1. 0. 1. 0. 0. 0.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]


In [ ]:
test_threshold = 6.058

def evaluate_paper(paper_idx):
    model_temp = blank_copy(model)
    model_temp.stdp = False
    model_temp.neuron_thresholds[:nlabels] = [test_threshold] * nlabels
    for t in range(2):
        model_temp.add_spike(t, paper_neurons[paper_idx], strong_connection)
    model_temp.simulate(1 + 2)
    output_spikes = np.sum(model_temp.ispikes[-(1):, :nlabels], axis=0)
    return output_spikes

In [ ]:
# single-threaded (slow, but better for debugging)
results = [evaluate_paper(paper_idx) for paper_idx in tqdm(test_idxs)]

  0%|          | 0/662 [00:00<?, ?it/s]

In [ ]:
from tabulate import tabulate

tp, fp, tn, fn = 0, 0, 0, 0
correct = 0
total = len(test_idxs)

for actual_idx, spikes in zip(test_idxs, results):
    correct_label = papers[actual_idx].label
    guesses = {lbl_neurons.inverse[idx][0] for idx, spiked in enumerate(spikes) if spiked}
    tp += correct_label in guesses
    fn += not bool(guesses)  # +1 false negative if no guesses
    fp += len([x for x in guesses if x != correct_label])
    correct += correct_label in guesses and len(guesses) == 1

n_guesses = total - fn
print(f"tp: {tp} / {n_guesses} / {total} | Perfect: {correct} / {n_guesses} / {total} (correct / attempted / total)")
print(f"tp:        {tp / total:>.6f} | Perfect: {correct / total:>.6f} | F1:        {tp / (tp + (0.5 * (fp + fn))):>.6f}")
print(f"Precision: {tp / (tp + fp):>.6f} | Recall:  {tp / (tp + fn):>.6f} | Accuracy:  {(tp + tn) / (tp + tn + fp + fn):>.6f}")

tabulate([[correct, tp, fp, tn, fn, n_guesses, total,
           model.stdp_Apos.tolist(), model.stdp_Aneg.tolist(), lbl_threshold,
           strong_connection, weak_connection, unknown_connection, test_threshold]], tablefmt="html")


tp: 491 / 589 / 662 | Perfect: 362 / 589 / 662 (correct / attempted / total)
tp:        0.741692 | Perfect: 0.546828 | F1:        0.740015
Precision: 0.643512 | Recall:  0.870567 | Accuracy:  0.587321


AttributeError: 'NeuromorphicModel' object has no attribute 'stdp_Apos'

In [ ]:
lbl_neurons

{'HCI': 0, 'AI': 1, 'Agents': 2, 'ML': 3, 'DB': 4, 'IR': 5}